In [1]:
import pandas as pd
import numpy as np
import yaml
from tqdm import tqdm
from glob import glob
from pathlib import Path
import textdistance

from gensim.utils import deaccent
import ftfy

pd.set_option('max_columns', None)

In [2]:
def clean_text(s):
    if not isinstance(s, str):
        return s
    
    s = deaccent(ftfy.fix_text(s))    
    try:
        s = s.strip().upper().encode('cp1252').decode().lower()
        s = deaccent(s)
    except Exception as e:
        s = s.lower()
        
    return s


def find_closest_match(codes, adm0, adm1, adm2, max_allowed_distance=3):
    sub_codes = codes[codes['adm0'] == adm0].copy()
    sub2_codes = sub_codes[np.logical_and(sub_codes['adm1_name'] == adm1, 
                                          sub_codes['adm2'] != 'NA')].copy()
    if sub2_codes.shape[0] > 0:
        sub_codes = sub2_codes.copy()
        s = adm2
        col = 'adm2_name'
    else:
        s = adm1
        col = 'adm1_name'
        
    sub_codes['fuzzy_match_on'] = col
    
    sub_codes['distance'] = sub_codes[col].apply(lambda x: textdistance.damerau_levenshtein(x, s))
    min_distance = sub_codes['distance'].min()
    
    if min_distance > max_allowed_distance:
        return None

    sub_codes = sub_codes[sub_codes['distance'] == min_distance]
    
    if sub_codes.shape[0] == 1:
        return sub_codes.iloc[0]
    elif sub_codes.shape[0] == 0:
        return None
    else:
        sub_codes['distance'] = sub_codes[col].apply(lambda x: textdistance.mra(x, s))
        return sub_codes.iloc[sub_codes['distance'].argmin()]
        
    return sub_codes

In [3]:
df = pd.read_csv('../Data/Monash/NL_Subnat.csv', encoding='cp1252')
print(df.shape)
df.head()

(876531, 8)


,objectid,countrycode,countryname,name_1,name_2,year,mean_light,sum_light
0,1,AFG,Afghanistan,Badakhshan,Baharak,1992,0.0,0
1,1,AFG,Afghanistan,Badakhshan,Baharak,1993,0.0,0
2,1,AFG,Afghanistan,Badakhshan,Baharak,1994,0.0,0
3,1,AFG,Afghanistan,Badakhshan,Baharak,1995,0.0,0
4,1,AFG,Afghanistan,Badakhshan,Baharak,1996,0.0,0


In [4]:
print(df.isnull().sum())
df.fillna('NA', inplace=True)

objectid           0
countrycode        0
countryname        0
name_1             0
name_2         17930
year               0
mean_light         0
sum_light          0
dtype: int64


In [5]:
codes = pd.read_csv('../Data/GADM_codes.csv')
codes.fillna('NA', inplace=True)

for col in ['name_1', 'name_2']:
    df[col + '_clean'] = df[col].apply(clean_text)
    
for col in ['adm1_name', 'adm2_name']:
    codes[col] = codes[col].apply(clean_text)
    
codes.head()

,adm0,adm0_name,adm1,adm1_name,adm2,adm2_name
0,ABW,Aruba,NA,na,NA,na
1,AFG,Afghanistan,NA,na,NA,na
2,AGO,Angola,NA,na,NA,na
3,AIA,Anguilla,NA,na,NA,na
4,ALA,Åland,NA,na,NA,na


In [6]:
keep_cols = ['objectid', 'adm0', 'adm0_name', 'adm1', 'adm1_name', 'adm2', 'adm2_name']

locs = df.drop_duplicates(['countrycode', 'name_1_clean', 'name_2_clean'])
print(locs.shape)

locs_full = locs.merge(codes, 
         left_on=['countrycode', 'name_1_clean', 'name_2_clean'], 
         right_on=['adm0', 'adm1_name', 'adm2_name'],
         how='left',
         indicator=True)
print(locs_full.shape)
print((locs_full['_merge'] == 'left_only').sum())

locs_adm2 = locs[locs['name_2'] != 'NA'].merge(codes, 
         left_on=['countrycode', 'name_2_clean'], 
         right_on=['adm0', 'adm2_name'],
         how='left',
         indicator=True)
locs_adm2.drop_duplicates('objectid', inplace=True)
print((locs_adm2['_merge'] == 'left_only').sum())

locs_full = locs_full.merge(locs_adm2[keep_cols], on='objectid', how='left', suffixes=['', '_y'])
mask = locs_full['_merge'] == 'left_only'
for col in keep_cols[1:]:
    locs_full[col] = np.where(mask, locs_full[col + '_y'], locs_full[col])
locs_full.drop(columns=[x + '_y' for x in keep_cols[1:]], inplace=True)
print((locs_full['adm0'].isnull()).sum())

(39857, 10)
(39862, 17)
13328
8440
8669


In [7]:
adm0 = {x: x for x in codes['adm0'].unique()}
mask = locs_full['adm0'].isnull()

adm0.update({
    'ZAR': 'COD',
    'KO-': 'XKO',
    'YUG': 'SRB',
    'TMP': 'TLS',
    'ANT': 'ANT',
    'ROM': 'ROU'
})

locs_full.loc[mask, 'adm0'] = locs_full.loc[mask, 'countrycode'].apply(lambda x: adm0.get(x, np.nan))

In [8]:
locs_full['fuzzy_match_on'] = np.nan
replace_cols = ['adm0', 'adm0_name', 'adm1', 'adm1_name', 'adm2', 'adm2_name', 'fuzzy_match_on']

mask = np.logical_or(locs_full['adm2'].isnull(), locs_full['adm2'].str.lower() == 'na')
not_found = 0
for idx, row in tqdm(locs_full[mask].iterrows(), total=mask.sum()):
    match = find_closest_match(codes, row['adm0'], row['name_1_clean'], row['name_2_clean'])

    if match is not None:
        for col in replace_cols:
            locs_full.loc[idx, col] = match[col]
    else:
        not_found += 1

print(f'{not_found} locations not found')

100%|██████████| 9255/9255 [02:39<00:00, 58.08it/s] 

2950 locations not found


# Get missing edo

In [9]:
edo = locs_full.drop_duplicates(['countrycode', 'name_1_clean'])
mask = np.logical_or(edo['adm1'].isnull(), edo['adm1'].str.lower() == 'na')
print(mask.sum())
edo = edo.loc[mask, ['countrycode', 'countryname', 'name_1_clean']].copy()

edo = edo.merge(codes[np.logical_and(codes['adm1'] != 'NA', codes['adm2'] == 'NA')],
         left_on=['countrycode', 'name_1_clean'], right_on=['adm0', 'adm1_name'],
         how='left')

edo['fuzzy_match_on'] = np.nan
edo['name_2_clean'] = 'NA'
replace_cols = ['adm1', 'adm1_name', 'fuzzy_match_on']

mask = np.logical_or(edo['adm1'].isnull(), edo['adm1'].str.lower() == 'na')
not_found = 0
for idx, row in tqdm(edo[mask].iterrows(), total=mask.sum()):
    match = find_closest_match(codes, row['countrycode'], row['name_1_clean'], row['name_2_clean'])

    if match is not None:
        for col in replace_cols:
            edo.loc[idx, col] = match[col]
    else:
        not_found += 1

print(f'{not_found} locations not found')

  7%|▋         | 16/241 [00:00<00:01, 157.20it/s]

301


100%|██████████| 241/241 [00:01<00:00, 166.35it/s]

226 locations not found


In [10]:
edo[edo['adm0'].isnull()].to_excel('../Output/NL/missing_edo.xlsx', index=False)

In [11]:
edo = edo.loc[edo['adm0'].notnull(), ['countrycode', 'name_1_clean', 'adm0', 'adm1']].drop_duplicates()
locs_full = locs_full.merge(edo, 
                            on=['countrycode', 'name_1_clean'], 
                            suffixes=['', '_y'], 
                            how='left')
mask = locs_full['adm1'].isnull()
for col in ['adm0', 'adm1']:
    locs_full.loc[mask, col] = locs_full.loc[mask, col + '_y']
    locs_full.drop(columns=col + '_y', inplace=True)
    
locs_full['fuzzy_match_on'].fillna(False, inplace=True)

In [12]:
locs_full.to_excel('../Output/NL/locs_full.xlsx', index=False)

# Matched edo

In [13]:
locs_full = pd.read_excel('../Output/NL/locs_full.xlsx')
codes = pd.read_csv('../Data/GADM_codes.csv')
codes.fillna('NA', inplace=True)

for col in ['name_1', 'name_2']:
    df[col + '_clean'] = df[col].apply(clean_text)
    
for col in ['adm1_name', 'adm2_name']:
    codes[col] = codes[col].apply(clean_text)

adm0 = {x: x for x in codes['adm0'].unique()}
mask = locs_full['adm0'].isnull()

adm0.update({
    'ZAR': 'COD',
    'KO-': 'XKO',
    'YUG': 'SRB',
    'TMP': 'TLS',
    'ANT': 'ANT',
    'ROM': 'ROU'
})

locs_full.loc[mask, 'adm0'] = locs_full.loc[mask, 'countrycode'].apply(lambda x: adm0.get(x, np.nan))

print(locs_full['adm0'].isnull().sum())
print(locs_full['adm1'].isnull().sum())

0
1665


In [14]:
edo = pd.read_excel('../Output/NL/missing_edo_v2.xlsx')

edo.drop(columns='countryname', inplace=True)
edo.drop_duplicates(inplace=True)

locs_full = locs_full.merge(edo, 
                            on=['countrycode', 'name_1_clean'], 
                            suffixes=['', '_y'], 
                            how='left',
                            validate='m:1')
mask = locs_full['adm1'].isnull()
for col in ['adm1']:
    locs_full.loc[mask, col] = locs_full.loc[mask, col + '_y']
    
locs_full['fuzzy_match_on'].fillna(False, inplace=True)
locs_full.drop(columns=locs_full.filter(regex='.*_y').columns, inplace=True)
locs_full.drop(columns=['_merge', 'adm0_name', 'adm1_name', 'adm2_name'], inplace=True)
    
print(locs_full['adm0'].isnull().sum())
print(locs_full['adm1'].isnull().sum())
    
locs_full.head()

0
586


,objectid,countrycode,countryname,name_1,name_2,year,mean_light,sum_light,name_1_clean,name_2_clean,adm0,adm1,adm2,fuzzy_match_on
0,1,AFG,Afghanistan,Badakhshan,Baharak,1992,0.0,0,badakhshan,baharak,AFG,AFG.1_1,AFG.1.1_1,False
1,2,AFG,Afghanistan,Badakhshan,Darwaz,1992,0.0,0,badakhshan,darwaz,AFG,AFG.1_1,AFG.1.2_1,False
2,3,AFG,Afghanistan,Badakhshan,Fayz abad,1992,0.0,0,badakhshan,fayz abad,AFG,AFG.13_1,AFG.13.3_1,False
3,4,AFG,Afghanistan,Badakhshan,Ishkashim,1992,0.0,0,badakhshan,ishkashim,AFG,AFG.1_1,AFG.1.4_1,False
4,5,AFG,Afghanistan,Badakhshan,Jurm,1992,0.0,0,badakhshan,jurm,AFG,AFG.1_1,AFG.1.5_1,False


In [15]:
locs_full = locs_full.merge(codes[['adm0', 'adm1', 'adm1_name']].drop_duplicates(), 
                            on=['adm0', 'adm1'], how='left')
locs_full.head()

,objectid,countrycode,countryname,name_1,name_2,year,mean_light,sum_light,name_1_clean,name_2_clean,adm0,adm1,adm2,fuzzy_match_on,adm1_name
0,1,AFG,Afghanistan,Badakhshan,Baharak,1992,0.0,0,badakhshan,baharak,AFG,AFG.1_1,AFG.1.1_1,False,badakhshan
1,2,AFG,Afghanistan,Badakhshan,Darwaz,1992,0.0,0,badakhshan,darwaz,AFG,AFG.1_1,AFG.1.2_1,False,badakhshan
2,3,AFG,Afghanistan,Badakhshan,Fayz abad,1992,0.0,0,badakhshan,fayz abad,AFG,AFG.13_1,AFG.13.3_1,False,jawzjan
3,4,AFG,Afghanistan,Badakhshan,Ishkashim,1992,0.0,0,badakhshan,ishkashim,AFG,AFG.1_1,AFG.1.4_1,False,badakhshan
4,5,AFG,Afghanistan,Badakhshan,Jurm,1992,0.0,0,badakhshan,jurm,AFG,AFG.1_1,AFG.1.5_1,False,badakhshan


In [16]:
replace_cols = ['adm0', 'adm0_name', 'adm1', 'adm1_name', 'adm2', 'adm2_name', 'fuzzy_match_on']

mask = np.logical_or(locs_full['adm2'].isnull(), locs_full['adm2'].str.lower() == 'na')
mask = np.logical_and(mask, locs_full['adm1_name'].notnull())

not_found = 0
for idx, row in tqdm(locs_full[mask].iterrows(), total=mask.sum()):
    match = find_closest_match(codes, row['adm0'], row['adm1_name'], row['name_2_clean'])

    if match is not None:
        for col in replace_cols:
            locs_full.loc[idx, col] = match[col]
    else:
        not_found += 1

print(f'{not_found} locations not found')

100%|██████████| 5249/5249 [00:50<00:00, 104.69it/s]

2307 locations not found


In [18]:
locs_full[locs_full['adm1_name'].isnull()]

,objectid,countrycode,countryname,name_1,name_2,year,mean_light,sum_light,name_1_clean,name_2_clean,adm0,adm1,adm2,fuzzy_match_on,adm1_name,adm0_name,adm2_name
1160,1162,AUS,Australia,New South Wales,Armidale Dumaresq Bal,1992,0.130806,552,new south wales,armidale dumaresq bal,AUS,NaN,NaN,False,NaN,NaN,NaN
1165,1167,AUS,Australia,New South Wales,Bankstown - North-West,1992,63.000000,1890,new south wales,bankstown - north-west,AUS,NaN,NaN,False,NaN,NaN,NaN
1166,1168,AUS,Australia,New South Wales,Bankstown - South,1992,58.571430,1640,new south wales,bankstown - south,AUS,NaN,NaN,False,NaN,NaN,NaN
1167,1169,AUS,Australia,New South Wales,Bankstown - North-East,1992,63.000000,1008,new south wales,bankstown - north-east,AUS,NaN,NaN,False,NaN,NaN,NaN
1168,1170,AUS,Australia,New South Wales,Bathurst Regional - Pt A,1992,10.041152,2440,new south wales,bathurst regional - pt a,AUS,NaN,NaN,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39684,52485,ROM,Romania,Covasna,NaN,1992,0.704918,2494,covasna,na,ROU,ROM.16_1,NaN,False,NaN,NaN,NaN
39689,52490,ROM,Romania,Gorj,NaN,1992,0.552021,3141,gorj,na,ROU,ROM.21_1,NaN,False,NaN,NaN,NaN
39690,52491,ROM,Romania,Harghita,NaN,1992,0.693350,4859,harghita,na,ROU,ROM.22_1,NaN,False,NaN,NaN,NaN
39692,52493,ROM,Romania,Ialomita,NaN,1992,0.638346,2810,ialomita,na,ROU,ROM.25_1,NaN,False,NaN,NaN,NaN
